# Redfish schemas: Why should I care

Powered by [HPE DEV Team](hpedev.io)

### Speakers : François Donzé


Version 0.1

<img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 125px;"/> 


## Introduction

This Jupyter notebook defines environment variables that will be used through the rest of the notebook. Then, it explains how Redfish schemas can help you in your development. For didactic reasons, commands presented in this notebook are not optimized.

## Create environment variables

The following `bash` code defines environment variables (i.e. IP address, username, password....) depending on your student ID number stored in variable `$Stud`. It creates as well a `.json` file containing the credentials of your OpenBMC appliance required to open a Redfish session. 

Click in the cell below and then click on the
<img src="Pictures/RunButton.png" style="display:inline;width=55px;height=25px"/>
icon to create the environment variables and the json file.

In [1]:
# Set Student ID number
Stud="00"
echo "You are Student $Stud"

# Create BMC location/ports variables
BmcIP=openbmcs:443${Stud}          # OpenBMC simulator
BmcURI="https://${BmcIP}"

# BMC Administrator credentials
BmcUser="student"
BmcPassword='P@ssw0rd!'

# Minimum required Redfish headers
HeaderODataVersion="OData-Version: 4.0"
HeaderContentType="Content-Type: application/json"

# Data files
ResponseHeaders="ResponseHeaders.txt"   # Used to hold HTTP response headers
SessionData="./CreateSession-data.json" # Body/Workload used to create the Redfish session
cat > ${SessionData} << __EOF__
{
        "UserName": "$BmcUser",
        "Password": "$BmcPassword"
}
__EOF__


# Verify we can reach the remote Bmc
curl --insecure --silent \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --request GET ${BmcURI}/redfish | jq &>>/dev/null || echo "WARNING: Problem reaching the remote BMC"
  


You are Student 00


## Retrieve the Redfish Service Entry point content (Root)

The Redfish Service Entry point is **`/redfish/v{RedfishVersion}/`**.

Run the next cell to retrieve Redfish version(s) available today in your OpenBMC.

This request does not require any authentication.

If you are not familiar with cURL, get help from its [manual page](https://curl.haxx.se/docs/manpage.html).

## Create a Redfish session using cURL


All the URIs below the Root entry point requires authentication. In this section you'll go through the session authentication method as it may differ from other Rest APIs (i.e. OneView). 

The following `curl` command sends a `POST` request toward the standard `/redfish/v1/SessionService/Sessions` URI of your  BMC. The body/workload of this request is in the `@${SessionData}` json file populated in the very first `bash` cell of this notebook (Environment variables). You can view its content by clicking on it from the left pane of your Jupyter environment.

Select and run the following cell.

If this `POST` request is successful, the BMC sends back a `Token` and a `Session Location` in the **headers of the response**. Response headers are saved in the `$ResponseHeaders` text file now present in the left pane of your Jupyter environment.


In [20]:
echo 'Create Session and print body response:' 

curl --dump-header  $ResponseHeaders \
     --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --request POST --data "@$SessionData" \
     ${BmcURI}/redfish/v1/SessionService/Sessions | jq
     
BmcToken=$(awk '/X-Auth-Token/ {print $NF}' $ResponseHeaders | tr -d '\r')
BmcSessionLocation="$BmcURI"$(awk '/^Loca.*Se/ {gsub("https://.*/red", "/red", $NF);print $NF}' $ResponseHeaders | tr -d '\r')

echo "Bmc Token: $BmcToken"
echo -e "Bmc Session Location: $BmcSessionLocation\n"

Create Session and print body response:
{
  "@odata.id": "/redfish/v1/SessionService/Sessions/yIKuKh6vDk",
  "@odata.type": "#Session.v1_0_2.Session",
  "Description": "Manager User Session",
  "Id": "yIKuKh6vDk",
  "Name": "User Session",
  "UserName": "student"
}
Bmc Token: nOhnx0jdxfNj0I6kw7hW
Bmc Session Location: https://openbmcs:44300/redfish/v1/SessionService/Sessions/yIKuKh6vDk



The following cell extracts the name of the BMCs present in your system and then, for each BMC it extracts its properties.

Run it and review the properties returned by your OpenBMC. Among them you should notice the `Actions` and the `Oem` resources which need some explanation. 

The `Actions` collection contains all the possible actions that can be performed on your BMC; With this version of OpenBMC, you can perform a reset of the BMC by posting the value `GracefulRestart` at `/redfish/v1/Managers/bmc/Actions/Manager.Reset`. You'll do this later.

The `Oem` collection contains resources specific to `OpenBmc` and not part of the Redfish standard. This is a way to allow computer makers to expose their specific and added value resources to the Rest API.

In [21]:
BmcList=$(curl --insecure --silent --noproxy "localhost, 127.0.0.1"  \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --header "X-Auth-Token: $BmcToken" \
  --request GET ${BmcURI}/redfish/v1/Managers | jq '.Members[]' | \
  awk -F/ '/@odata.id/ {print $NF}' | tr -d '"' )
  
echo "List of BMC(s) present in this system:"
echo -e "$BmcList\n"

for bmc in $BmcList ; do   
    echo "Properties of BMC: $bmc" 
    curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
      --header  "$HeaderContentType" --header "$HeaderODataVersion" \
      --header "X-Auth-Token: $BmcToken" \
      --request GET ${BmcURI}/redfish/v1/Managers/${bmc} | jq
done


List of BMC(s) present in this system:
bmc

Properties of BMC: bmc
{
  "@odata.id": "/redfish/v1/Managers/bmc",
  "@odata.type": "#Manager.v1_3_0.Manager",
  "Actions": {
    "#Manager.Reset": {
      "ResetType@Redfish.AllowableValues": [
        "GracefulRestart"
      ],
      "target": "/redfish/v1/Managers/bmc/Actions/Manager.Reset"
    }
  },
  "DateTime": "2020-07-24T16:40:18+00:00",
  "Description": "Baseboard Management Controller",
  "EthernetInterfaces": {
    "@odata.id": "/redfish/v1/Managers/bmc/EthernetInterfaces"
  },
  "FirmwareVersion": "2.8.0-dev-1427-g64e281927",
  "GraphicalConsole": {
    "ConnectTypesSupported": [
      "KVMIP"
    ],
    "MaxConcurrentSessions": 4,
    "ServiceEnabled": true
  },
  "Id": "bmc",
  "Links": {
    "ManagerForChassis": [
      {
        "@odata.id": "/redfish/v1/Chassis/chassis"
      }
    ],
    "ManagerForChassis@odata.count": 1,
    "ManagerForServers": [
      {
        "@odata.id": "/redfish/v1/Systems/system"
      }
    ],
 

If you want to view the network protocols supported by your BMC, you can retrieve them with a `GET` of the `NetworkProtocol` URI mentionned in the output of the above `GET` request.

Run the next `curl`command. Its output should show an empty array of `NTPServers` (if not, contact your instructor). It contains as well the **Type** of the resources in this sub-tree: `@odata.type = #ManagerNetworkProtocol.v1_4_0.ManagerNetworkProtocol`.

Said differently, the `NetworkProtocol` resources falls in the **`ManagerNetworkProtocol`**. You will need this info later.


In [ ]:
echo "Network Protocol configuration:"
curl --insecure --silent --noproxy "localhost, 127.0.0.1"  \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --header "X-Auth-Token: $BmcToken" \
  --request GET ${BmcURI}/redfish/v1/Managers/${bmc}/NetworkProtocol | jq


As it is always good to have the correct date and time in a BMC, you may want to supply at least one server IP in the `NTPServers` array of your BMC. To reach that goal you have first to verify in the Redfish Schema whether the `NTPServers` array can be modified. 

Generally speaking the location of the Redfish Schemas of a particular OData type is under the `/redfish/v1/JsonSchemas/{type}` endpoint.

Run the following command listing the location(s) of the `ManagerNetworkProtocol` schema used by your BMC and study its output. 

The **`PublicationUri`** URI requires an Internet connection to reach `http://redfish.dmtf.org`.

However, the `URI` pointer  does not require any Internet access to view its content as it is embedded in the BMC at `/redfish/v1/JsonSchemas/ManagerNetworkProtocol/ManagerNetworkProtocol.json` 

In [ ]:
echo "Manager Network Protocol schema locations:"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $BmcToken" \
     --request GET \
     ${BmcURI}/redfish/v1/JsonSchemas/ManagerNetworkProtocol | jq

Using the embedded `URI` You can retrieve the definition of the `NTPServers` object and verify you will be able to modify it. 

Run the following `curl` command which extracts the `NTPServers` schema definition and note the **`readonly = false`** property.

In [ ]:
echo "NTPServers schema definition:"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $BmcToken" \
     --request GET \
     ${BmcURI}/redfish/v1/JsonSchemas/ManagerNetworkProtocol/ManagerNetworkProtocol.json | \
     jq '.definitions | .NTPProtocol | .properties | .NTPServers' 

You are now sure that it is possible to alter/populate the list of `NTPServers` in your BMC. 

The following commands performs a `PATCH` of the `NetworkProtocol` endpoint with a single NTP server IP address.

This `PATCH` request does not return any response data. Other Redfish implementation (i.e. HPE iLO) are more verbose. However, by checking the response header file `$ResponseHeaders`, you should see an `HTTP/1.1: 204` return code stating that the request was successful.

In [ ]:
echo "Patching NTP Servers"
curl --dump-header  $ResponseHeaders \
     --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $BmcToken" \
     --request PATCH --data '{ "NTP": {"NTPServers": ["192.168.0.99", ""]} }' \
     ${BmcURI}/redfish/v1/Managers/bmc/NetworkProtocol | jq

Verify with the following command that the `NTPServers` list contains the IP address you supplied.

In [ ]:
echo "NTP Server list:"
curl --insecure --silent --noproxy "localhost, 127.0.0.1"  \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --header "X-Auth-Token: $BmcToken" \
  --request GET ${BmcURI}/redfish/v1/Managers/${bmc}/NetworkProtocol | jq '.NTP'

## Perform an action: Reset OpenBMC

In the previous section you modified a resource that is not requiring a reset of the BMC to be taken into account. However other parameters may require a restart when changed.

In this paragraph you will perform the `GracefulRestart` action seen previously in your OpenBMC using a `POST` request toward the corresponding target. 

After you run this reset command, run the next `bash` cell in order to wait until the BMC is back online.

In [ ]:
echo "Starting a reset of the BMC at" ; date
echo

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $BmcToken" \
     --request POST --data '{ "ResetType": "GracefulRestart"}' \
     ${BmcURI}/redfish/v1/Managers/bmc/Actions/Manager.Reset | jq

## Wait until OpenBMC is back online

The following cell loops until the BMC returns a valid output to a `GET` request. Run it and wait until the BMC is back on line. This should take about two minutes.

In [ ]:
ret=""
while [ "X${ret}" != "X0" ] ; do
    timeout 3 curl --insecure --noproxy "localhost, 127.0.0.1"  --silent \
      --header  "$HeaderContentType" --header "$HeaderODataVersion" \
      --header "X-Auth-Token: $BmcToken" \
      --request GET ${BmcURI}/redfish/v1/Managers/$bmc > /dev/null
    ret=$?
done

echo "BMC is back online at " ; date 
echo

## Delete sessions

It is extremely important to delete Redfish sessions to avoid reaching the maximum number of opened sessions in a BMC, preventing any access to it. Read this [article](https://developer.hpe.com/blog/managing-ilo-sessions-with-redfish) for more detail.

In [ ]:
echo "Body response of a session deletion:"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" --header "X-Auth-Token: $BmcToken" \
     --request DELETE $BmcSessionLocation | jq

### What next ?

If you want to re-run this notebook against an **HPE iLO 5**, from your Jupyter Home page, you can duplicate it and then modify the **`BmcIP`** variable with **`16.31.87.207`**.

Then, you will be able to compare the output of OpenBMC and HPE iLO 5 Redfish implementations.

 It is time now to go through the **[Lab 3 notebook](3-Aspire-RedfishPython.ipynb)** to study a Python code suitable for several Redfish implementation

<img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/><img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 50px;"/>